# Calculate effective temperature for Eelde/Groningen

## Libraries

In [40]:
from pandas import DataFrame, read_csv
import datetime
import math
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import pylab
from scipy.optimize import curve_fit
import os

## Set system parameters
### Working directory

In [41]:
working_directory ='/Users/peter/python/pyBuurtwarmte'
os.chdir(working_directory)

## Read KNMI data from Eelde airport


### KNMI format

```
# BRON: KONINKLIJK NEDERLANDS METEOROLOGISCH INSTITUUT (KNMI)
# Opmerking: door stationsverplaatsingen en veranderingen in waarneemmethodieken zijn deze tijdreeksen van dagwaarden mogelijk inhomogeen! Dat betekent dat deze reeks van gemeten waarden niet geschikt is voor trendanalyse. Voor studies naar klimaatverandering verwijzen we naar de gehomogeniseerde reeks maandtemperaturen van De Bilt <http://www.knmi.nl/kennis-en-datacentrum/achtergrond/gehomogeniseerde-reeks-maandtemperaturen-de-bilt> of de Centraal Nederland Temperatuur <http://www.knmi.nl/kennis-en-datacentrum/achtergrond/centraal-nederland-temperatuur-cnt>.
# 
# 
# STN      LON(east)   LAT(north)     ALT(m)  NAME
# 280:         6.585       53.125       5.20  EELDE
# 
# YYYYMMDD = Date (YYYY=year MM=month DD=day); 
# FG       = Daily mean windspeed (in 0.1 m/s); 
# TG       = Daily mean temperature in (0.1 degrees Celsius); 
# Q        = Global radiation (in J/cm2); 
# 
# STN,YYYYMMDD,   FG,   TG,    Q
# 
  280,20120101,   63,  108,  110
```
#### Clean and scrub data
1. skip first 12 rows
1. remove extra whitespaces from strings (because they remain in headings)
1. define YYYYMMDD data as string
1. drop the empty data after the headings and re-index data frame

In [42]:
#if KNMI_Teff is not None:
#    KNMI_Teff = KNMI_Teff.iloc[0:0]
#    del KNMI_Teff

In [43]:
file = "KNMI_2012_2017.txt"
KNMI_Teff = pd.read_csv(file, 
                        skiprows=12, 
                        skipinitialspace=True, 
                        dtype={'YYYYMMDD': str})
KNMI_Teff.reset_index(drop=True)

,# STN,YYYYMMDD,FG,TG,Q
0,280,20120101,63,108,110
1,280,20120102,53,64,191
2,280,20120103,106,64,28
3,280,20120104,95,62,215
4,280,20120105,113,67,149
5,280,20120106,73,62,263
6,280,20120107,72,69,108
7,280,20120108,54,66,237
8,280,20120109,43,72,90
9,280,20120110,40,55,188


## Prepare data
1. add new column 'Date' and convert string date to machine readable datetime
1. make Date new index (N.B.: required for fast search of dates with .loc)

In [44]:
KNMI_Teff['YYYYMMDD']

0       20120101
1       20120102
2       20120103
3       20120104
4       20120105
5       20120106
6       20120107
7       20120108
8       20120109
9       20120110
10      20120111
11      20120112
12      20120113
13      20120114
14      20120115
15      20120116
16      20120117
17      20120118
18      20120119
19      20120120
20      20120121
21      20120122
22      20120123
23      20120124
24      20120125
25      20120126
26      20120127
27      20120128
28      20120129
29      20120130
          ...   
2162    20171202
2163    20171203
2164    20171204
2165    20171205
2166    20171206
2167    20171207
2168    20171208
2169    20171209
2170    20171210
2171    20171211
2172    20171212
2173    20171213
2174    20171214
2175    20171215
2176    20171216
2177    20171217
2178    20171218
2179    20171219
2180    20171220
2181    20171221
2182    20171222
2183    20171223
2184    20171224
2185    20171225
2186    20171226
2187    20171227
2188    20171228
2189    201712

In [45]:
# add column 'Date' for machine readable date and set and re-index
KNMI_Teff['Date'] = KNMI_Teff['YYYYMMDD'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
KNMI_Teff.set_index('Date', inplace=True)
KNMI_Teff.reset_index(drop=True)

,# STN,YYYYMMDD,FG,TG,Q
0,280,20120101,63,108,110
1,280,20120102,53,64,191
2,280,20120103,106,64,28
3,280,20120104,95,62,215
4,280,20120105,113,67,149
5,280,20120106,73,62,263
6,280,20120107,72,69,108
7,280,20120108,54,66,237
8,280,20120109,43,72,90
9,280,20120110,40,55,188


In [46]:
KNMI_Teff

,# STN,YYYYMMDD,FG,TG,Q
Date,,,,,
2012-01-01,280,20120101,63,108,110
2012-01-02,280,20120102,53,64,191
2012-01-03,280,20120103,106,64,28
2012-01-04,280,20120104,95,62,215
2012-01-05,280,20120105,113,67,149
2012-01-06,280,20120106,73,62,263
2012-01-07,280,20120107,72,69,108
2012-01-08,280,20120108,54,66,237
2012-01-09,280,20120109,43,72,90


## Calculate effective temperature
The effective average day temperature is calculated as:
$T_{eff, day} = \overline{T}_{day} - C_u \cdot \overline{u}_{wind, day} + C_I \cdot \overline{I}_{rad, day} $

Where $\overline{T}_{day}$ is given in $^\circ C$, $\overline{u}_{wind, day}$ is given in $\frac{m}{s}$, and $\overline{I}_{rad, day}$ is given in $\frac{J}{cm^2}$.
      
The constants $C_u$ and $C_I$ have the following values and dimsions: $C_u = \frac{2}{3} (\frac{^\circ C}{\frac{m}{s}})$ and $C_I = \frac{1}{40 \cdot 24} (\frac{^\circ C}{\frac{J}{cm^2}})$

In [47]:
KNMI_Teff['Teff'] = KNMI_Teff['TG']/10 - (2/3 * (KNMI_Teff['FG'] / 10)) + (1 / (40 * 24)) * KNMI_Teff['Q']

## Calculate home sensitive effective temperature
The home sensitive effective temperature takes the last 3 days before the actual date of the effecive temperature into account. In essence the "composite" effective temperature is a weighted average of current and past $T_{eff}$ such that
$T_{eff, composite} = 0.53 \cdot T_{eff, day-0} + 0.27 \cdot T_{eff, day-1} + 0.13 \cdot T_{eff, day-2} + 0.07 \cdot T_{eff, day-3}$ 

add another culumn to the data frame and fill it with effective temperature

In [48]:
KNMI_Teff['Teff_lag'] = KNMI_Teff['Teff']

In [49]:
#start_date = pd.to_datetime('2012-01-01')+ pd.DateOffset(days= 3)

for i in KNMI_Teff.index:
        Teff_lag = KNMI_Teff.loc[i, 'Teff']
        date_min_3  = (i + pd.DateOffset(days= -3))
        date_min_2  = (i + pd.DateOffset(days= -2))
        date_min_1  = (i + pd.DateOffset(days= -1))
        date_min_0  = (i + pd.DateOffset(days= +0))
#        print(i)
        if ((date_min_3 in KNMI_Teff.index) and (date_min_2 in KNMI_Teff.index) and (date_min_1 in KNMI_Teff.index) and (date_min_0 in KNMI_Teff.index)):
            Teff_lag       = 0.53 * KNMI_Teff.at[date_min_0,'Teff'] \
                           + 0.27 * KNMI_Teff.at[date_min_1,'Teff'] \
                           + 0.13 * KNMI_Teff.at[date_min_2,'Teff'] \
                           + 0.07 * KNMI_Teff.at[date_min_3,'Teff']
#            print(i, date_min_3, date_min_2, date_min_1, date_min_0)
#            print(0.53 * KNMI_Teff.at[date_min_0,'Teff'] \
#                           + 0.27 * KNMI_Teff.at[date_min_1,'Teff'] \
#                           + 0.13 * KNMI_Teff.at[date_min_2,'Teff'] \
#                           + 0.07 * KNMI_Teff.at[date_min_3,'Teff'] , Teff_lag)
            KNMI_Teff.loc[i,'Teff_lag'] = Teff_lag
        else:
            print("not enough past data for the following  date: ", i)

not enough past data for the following  date:  2012-01-01 00:00:00
not enough past data for the following  date:  2012-01-02 00:00:00
not enough past data for the following  date:  2012-01-03 00:00:00


In [57]:
#to check specific date, fill in date and uncomment in next line
#KNMI_Teff.loc['2017-12-02']
# to check data uncomment next line
#KNMI_Teff

In [58]:
from bokeh.models import ColumnDataSource, HoverTool, Range1d, LinearAxis
from bokeh.plotting import figure, show, output_file
from bokeh.io import show, output_notebook, show

output_notebook()

source = ColumnDataSource(KNMI_Teff)

fig = figure()

fig  = figure(x_axis_type="datetime")
# Define x-axis
#fig.xaxis.axis_type = "datetime"
fig.xaxis.axis_label = 'Date'

# Define 1st LHS y-axis
fig.yaxis.axis_label = 'Temperature [˚C]'
fig.y_range = Range1d(start=-15, end=30)

# Create 1st RHS y-axis
fig.extra_y_ranges['temp'] = Range1d(start=-15, end=30)
fig.add_layout(LinearAxis(y_range_name='temp', axis_label='Temperature [°C]'), 'right')

fig.line(
    'Date',
    'Teff',
    source = source,
    legend = 'Temperature eff',
    line_width = 0.5,
    color = 'grey'
)

fig.line(
    'Date',
    'Teff_lag',
    source = source,
    legend = 'Temperature cum',
    y_range_name = 'temp',
    line_width = 0.2,
    color = 'black'
)

fig.add_tools(HoverTool(tooltips=[("Teff", "@Teff"), ("Teff_lag", "@Teff_lag")]))
fig.toolbar_location = 'above'

show(fig)

Loading BokehJS ...

## Write KNMI_Teff results to CSV file

In [54]:
file = "KNMI_Teff_2012_2017.csv"
export_report = KNMI_Teff.to_csv(file)